In [1]:
import os

import tensorflow as tf
import pandas as pd
import numpy as np
    
from catanatron_gym.envs.catanatron_env import ACTION_SPACE_SIZE
from catanatron_gym.features import get_feature_ordering

2021-11-07 16:15:40.489482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-07 16:15:40.489506: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATASET_PATH = os.path.join("data/ab2-dataset", "main.csv.gzip")
BATCH_SIZE = 32
SHUFFLE_SEED = 1

rows_per_bytes = 6345 / 1209734
size = os.path.getsize(DATASET_PATH)  # bytes
estimated_rows = size * rows_per_bytes
print(estimated_rows)

258507.50417033827


In [3]:
data = pd.read_csv(DATASET_PATH, nrows=10, compression='gzip')
data

,F_BANK_BRICK,F_BANK_DEV_CARDS,F_BANK_ORE,F_BANK_SHEEP,F_BANK_WHEAT,F_BANK_WOOD,"F_EDGE(0, 1)_P0_ROAD","F_EDGE(0, 1)_P1_ROAD","F_EDGE(0, 20)_P0_ROAD","F_EDGE(0, 20)_P1_ROAD",...,BT_3692,BT_3693,BT_3694,BT_3695,ACTION,RETURN,DISCOUNTED_RETURN,TOURNAMENT_RETURN,DISCOUNTED_TOURNAMENT_RETURN,VICTORY_POINTS_RETURN
0,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,110,0.0,0.0,8.0,3.998696,7.944987
1,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,52,0.0,0.0,8.0,3.998696,7.944987
2,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,107,0.0,0.0,8.0,3.998696,7.944987
3,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,45,0.0,0.0,8.0,3.998696,7.944987
4,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987
5,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,8.0,3.998696,7.944987
6,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987
7,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,50,0.0,0.0,8.0,3.998696,7.944987
8,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,8.0,3.998696,7.944987
9,19.0,25.0,18.0,17.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987


In [4]:
# ===== Read Dataset
INPUT_COLUMNS = list(filter(lambda x: x.startswith("F_"), data.columns))
LABEL_COLUMN = "ACTION"
RETURN_COLUMN = "DISCOUNTED_RETURN"
INPUT_SHAPE = (len(INPUT_COLUMNS),)

dataset = tf.data.experimental.make_csv_dataset(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    label_name=LABEL_COLUMN,
    select_columns=INPUT_COLUMNS + [LABEL_COLUMN, RETURN_COLUMN],
    compression_type="GZIP",
    shuffle=False,
)

2021-11-07 16:11:07.949690: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-07 16:11:08.034267: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-07 16:11:08.034300: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bcollazo-Erazer-X510): /proc/driver/nvidia/version does not exist
2021-11-07 16:11:08.034688: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# ===== Select Top 10%
return_column_dataset = tf.data.experimental.make_csv_dataset(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    select_columns=[RETURN_COLUMN],
    compression_type="GZIP",
    shuffle=False,
)
print("read dataset")

def collect_returns(state, batch):
    features = batch
    return tf.concat([state, features['DISCOUNTED_RETURN']], 0)

initial_state = []
rewards = return_column_dataset.reduce(initial_state=initial_state, reduce_func=collect_returns)
print(rewards)

top_k = tf.math.top_k(rewards, k=10)
top_k.values.numpy()

In [6]:
# ===== Build Class Weights
def count(counts, batch):
    features, labels = batch
    for i in range(ACTION_SPACE_SIZE):
        class_i = labels == i
        class_i = tf.cast(class_i, tf.int32)
        counts[f"class_{i}"] += tf.reduce_sum(class_i)
    return counts

initial_state = {f"class_{i}": 0 for i in range(ACTION_SPACE_SIZE)}
counts = dataset.reduce(initial_state=initial_state, reduce_func=count)
counts = np.array([counts[f"class_{i}"].numpy() for i in range(ACTION_SPACE_SIZE)]).astype(np.float32)

fractions = counts/counts.sum()
weights = (1 / counts) * (counts.sum() / ACTION_SPACE_SIZE)
weights[weights == np.inf] = 0  # Drop inf to 0
class_weight = {i: weights[i] for i in range(ACTION_SPACE_SIZE)}
print(class_weight)

# def class_func(features, label):
#     return label

# n = ACTION_SPACE_SIZE
# target_dist = [1.0 / n for i in range(n)]
# resampler = tf.data.experimental.rejection_resample(
#     class_func, target_dist=target_dist)

{0: 0.010107901, 1: 0.7889809, 2: 0.56042695, 3: 0.7279772, 4: 0.6957024, 5: 0.7279772, 6: 0.6538314, 7: 0.6889151, 8: 1.5868268, 9: 2.0175369, 10: 2.0467765, 11: 1.3076628, 12: 2.139812, 13: 1.3323357, 14: 1.7435504, 15: 1.661501, 16: 2.393688, 17: 1.319884, 18: 2.8245516, 19: 2.4776769, 20: 0.2639768, 21: 2.2417078, 22: 1.9084809, 23: 1.6421812, 24: 1.8830345, 25: 2.0467765, 26: 2.2417078, 27: 2.1078744, 28: 2.4349585, 29: 2.5677743, 30: 2.5219212, 31: 1.787691, 32: 1.9614942, 33: 1.4265412, 34: 1.3579576, 35: 1.4265412, 36: 1.81061, 37: 1.4559544, 38: 1.7435504, 39: 1.2838871, 40: 1.1867865, 41: 1.8582577, 42: 1.1576031, 43: 1.6812809, 44: 1.2956659, 45: 1.16717, 46: 1.9084809, 47: 1.1867865, 48: 1.8582577, 49: 1.2956659, 50: 1.2498016, 51: 1.3076628, 52: 1.3323357, 53: 1.9084809, 54: 1.6233056, 55: 1.3323357, 56: 1.2498016, 57: 1.7435504, 58: 1.2174792, 59: 1.8582577, 60: 1.2174792, 61: 1.6233056, 62: 1.81061, 63: 5.649103, 64: 4.5557284, 65: 8.826724, 66: 4.1537523, 67: 6.419436, 

/tmp/ipykernel_537947/3114565155.py:15: RuntimeWarning: divide by zero encountered in true_divide
  weights = (1 / counts) * (counts.sum() / ACTION_SPACE_SIZE)


In [7]:
def preprocess(batch, label):
    features = tf.stack(
        [tf.cast(tensor, tf.float32) 
         for feature_name, tensor in batch.items() 
         if feature_name in INPUT_COLUMNS and feature_name != RETURN_COLUMN
        ], axis=1
    )
    return features, tf.stack(label)

dataset = dataset.map(preprocess)
    # .unbatch().apply(resampler).batch(BATCH_SIZE)\
    # .map(lambda extra_label, features_and_label: features_and_label)\
dataset

<MapDataset shapes: ((None, 614), (None,)), types: (tf.float32, tf.int32)>

In [8]:
inputs = tf.keras.Input(shape=INPUT_SHAPE)
dense = tf.keras.layers.Dense(64, activation="relu", dtype='float64')(inputs)
output_dense = tf.keras.layers.Dense(ACTION_SPACE_SIZE, dtype='float64')(dense)
model = tf.keras.Model(inputs=inputs, outputs=output_dense)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['categorical_accuracy']
)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 614)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                39360     
_________________________________________________________________
dense_1 (Dense)              (None, 290)               18850     
Total params: 58,210
Trainable params: 58,210
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(dataset, epochs=10)

Epoch 1/10
1280/1280 [==============================] - 26s 20ms/step - loss: 3.0008 - categorical_accuracy: 0.4991
Epoch 2/10
1280/1280 [==============================] - 22s 17ms/step - loss: 2.4822 - categorical_accuracy: 0.4921
Epoch 3/10
1280/1280 [==============================] - 21s 17ms/step - loss: 2.3044 - categorical_accuracy: 0.5270
Epoch 4/10
1280/1280 [==============================] - 22s 17ms/step - loss: 2.1451 - categorical_accuracy: 0.5843
Epoch 5/10
1280/1280 [==============================] - 21s 17ms/step - loss: 2.0310 - categorical_accuracy: 0.6130
Epoch 6/10
1280/1280 [==============================] - 20s 16ms/step - loss: 1.9534 - categorical_accuracy: 0.6206
Epoch 7/10
1280/1280 [==============================] - 19s 15ms/step - loss: 1.8970 - categorical_accuracy: 0.6224
Epoch 8/10
1280/1280 [==============================] - 18s 14ms/step - loss: 1.8521 - categorical_accuracy: 0.6231
Epoch 9/10
1280/1280 [==============================] - 19s 14ms/step - 

In [10]:
model.save("data/models/cross-entropy-model")

2021-10-17 08:43:51.680393: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: data/models/cross-entropy-model/assets
